In [42]:
from os.path import abspath
from pyspark.sql import SparkSession

# set default location for [spark-warehouse]
warehouse_location = abspath('spark-warehouse')


In [43]:
spark = (
    SparkSession
    .builder
    .appName("landing_to_bronze")
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.1.1,io.delta:delta-core_2.12:1.0.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.sql.debug.maxToStringFields","1000")
    .config("spark.kryoserializer.buffer.max","128m")
    .config("spark.driver.memory", "20G")
    .enableHiveSupport()
    .getOrCreate()
)
spark.sparkContext.setLogLevel("WARN")

In [44]:
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.project.id", "hering-datalake-prod")
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

---

In [45]:
from pyspark.sql.avro.functions import from_avro, to_avro
import pyspark.sql.functions as f
from urllib.request import urlopen
import json
from pyspark.sql.types import StringType, StructType
from dateutil import parser
from delta.tables import *
from loguru import logger
from datetime import datetime

In [46]:
bronze_path = 'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/ecommerce/bnsvher602/linx_db/dbo/loja_venda/'
silver_path = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/ecommerce/bnsvher602/linx_db/dbo/loja_venda/'
silver_path_restricted = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/ecommerce/bnsvher602/linx_db/dbo/loja_venda_restrito/'

In [47]:
def genereteSymlinkManifest(spark:SparkSession, bucket:str) -> None:
    """
    Gera o arquivo symlink manifest para a tabela Delta do bucket de referencia
    """
    delta_table = DeltaTable.forPath(spark, bucket)
    delta_table.generate("symlink_format_manifest")

def pathExists(path, spark:SparkSession):
    sc = spark.sparkContext
    fs = sc._jvm.org.apache.hadoop.fs.FileSystem.get(
        sc._jvm.java.net.URI.create("gs://" + path.split("/")[2]),
        sc._jsc.hadoopConfiguration(),
    )
    return fs.exists(sc._jvm.org.apache.hadoop.fs.Path(path))

def lastDate(delta_path:str, spark:SparkSession):
    if not pathExists(delta_path + 'job_control/last_date', spark):
        try:
            df_temp = (
                spark
                .read
                .format("delta")
                .load(delta_path)
            )

            list_result = (
                df_temp
                .agg({"timestamp_kafka": "max"}).collect()
            )

            last_date = str(list_result[0].asDict()['max(timestamp_kafka)'])
        except:
            last_date = ('01-01-2021 12:00AM')

        df = spark.read.json(spark.sparkContext.parallelize([{'last_date': last_date}]))
        df.write.mode('overwrite').csv(delta_path + 'job_control/last_date', header = 'true')
    else:
        df = spark.read.format("csv").load(delta_path + 'job_control/last_date', header = 'true')
        last_date = df.collect()[0].asDict()['last_date']

    logger.info('Last date: ' + last_date)

    return parser.parse(last_date)

def updateLastDate(delta_path:str, df:DataFrame, spark:SparkSession):
    list_result = (
        df
        .agg({"timestamp_kafka": "max"}).collect()
    )

    last_date = str(list_result[0].asDict()['max(timestamp_kafka)'])
    df = spark.read.json(spark.sparkContext.parallelize([{'last_date': last_date}]))
    df.write.mode('overwrite').csv(delta_path + 'job_control/last_date', header = 'true')

    return last_date


In [48]:
def getOperationsCounts(df:DataFrame, spark:SparkSession):
    op_count_list =(
        df
        .groupBy('__op')
        .count()
        .collect()
    )
    
    new_data_count = 0
    updating_data_count = 0
    deleting_data_count = 0

    if len(op_count_list) > 0:
        for i in op_count_list:
            dic_temp = i.asDict()
            if (dic_temp['__op'] == 'c' or dic_temp['__op'] == 'r'):
                new_data_count = new_data_count + dic_temp['count']
            elif (dic_temp['__op'] != 'c' and dic_temp['__op'] != 'r'):
                updating_data_count = updating_data_count + dic_temp['count']
            elif (dic_temp['__op'] == 'd'):
                deleting_data_count = deleting_data_count + dic_temp['count']

    new_total_data_count = new_data_count + updating_data_count + deleting_data_count
    
    return new_total_data_count, new_data_count, updating_data_count, deleting_data_count

In [49]:
def getData(spark:SparkSession, bucket:str, date_colunm_name:str, date_begin:datetime, source_format:str='delta'):
    """
    Carrega Dataframe filtrando os dados do bucket de referência conforme 
    data de início enviada no parametro

    :param str date_begin: Data de início para filtro dos dados
    """
    df = (
        spark
        .read
        .format(source_format)
        .load(bucket)
        .where(
            (f.col("year")>date_begin.year)|
              (
                (f.col("year")==date_begin.year)&
                (
                  (f.col("month")>date_begin.month)|
                  (
                    (f.col("month")==date_begin.month)&
                    (
                      (f.col("day")>date_begin.day)|
                      (
                        (f.col("day")==date_begin.day)&
                        (f.col("hour")>=date_begin.hour)&
                        (f.col(date_colunm_name)>date_begin)
                      )
                    )
                  )
                )          
              ) 
        )
    )
    return df

In [50]:
def writeNewData(spark:SparkSession, df:DataFrame, bucket:str, destination_format:str='delta'):
    df_r = df.where((f.col('__op') == 'r')|(f.col('__op')=='c'))
    (
        df_r
        .write
        .format(destination_format)
        .mode("append")
        .option("checkpointLocation", bucket + "checkpoint")
        .save(bucket)
    )
    
    return True

In [51]:
def updateNewData(spark:SparkSession, df:DataFrame, bucket:str, pk:list=['pk_single']):
    table = DeltaTable.forPath(spark, bucket)
    df_table_u = df.where((f.col('__op') != 'r')&(f.col('__op')!='c'))
    df_table_u_unique = df_table_u.orderBy('timestamp_kafka', ascending = False).dropDuplicates(pk)
    merge_string = 'destination.' + pk[0] + ' = updates.' + pk[0]
    (
        table.alias('destination')
        .merge(df_table_u_unique.alias('updates'), merge_string)
        .whenMatchedUpdateAll()
        .whenNotMatchedInsertAll()
        .execute()
    )
    
    return True

In [52]:
def deleteNewData(spark:SparkSession, bucket:str):
    table = DeltaTable.forPath(spark, bucket)
    table.delete(col('__op') == 'd')
    
    return True

In [53]:
# Reading last silver data
last_date = lastDate(silver_path, spark)

2022-03-22 12:33:37.488 | INFO     | __main__:lastDate:41 - Last date: 2022-03-19 21:48:24.407000


In [54]:
df_table = getData(spark, bronze_path, 'timestamp_kafka', last_date, 'delta')

In [55]:
new_total_data_count, new_data_count, updating_data_count, deleting_data_count = getOperationsCounts(df_table, spark)
logger.info('New Total Data:' + str(new_total_data_count))
logger.info('New Data:' + str(new_data_count))
logger.info('Updating Data:' + str(updating_data_count))
logger.info('Deleting Data:' + str(deleting_data_count))

2022-03-22 12:35:33.624 | INFO     | __main__:<module>:2 - New Total Data:21457 
2022-03-22 12:35:33.626 | INFO     | __main__:<module>:3 - New Data:21136
2022-03-22 12:35:33.627 | INFO     | __main__:<module>:4 - Updating Data:321
2022-03-22 12:35:33.628 | INFO     | __main__:<module>:5 - Deleting Data:0


In [56]:
if new_total_data_count > 0:
    logger.info('Defining restricted table')
    df_table_restricted = (
        df_table
        .select(f.col('CODIGO_FILIAL'), f.col('TICKET'), f.col('DATA_VENDA'), f.col('CODIGO_CLIENTE'), f.col('CPF_CGC_ECF'), f.col('__op'), f.col('timestamp_kafka'))
        .withColumn("CODIGO_CLIENTE_HASH", f.sha2(f.col('CODIGO_CLIENTE'), 256))    
        .withColumn("CPF_CGC_ECF_HASH", f.sha2(f.col('CPF_CGC_ECF'), 256))
        .withColumn('pk_single', f.concat(f.col('CODIGO_FILIAL'), f.col('TICKET'), f.col('DATA_VENDA')))
    )    

    logger.info('Defining hash table')
    df_table_hash = (
        df_table
        .withColumn("CODIGO_CLIENTE", f.sha2(f.col('CODIGO_CLIENTE'), 256))
        .withColumn("CPF_CGC_ECF_HASH", f.sha2(f.col('CPF_CGC_ECF'), 256))
        .withColumn('pk_single', f.concat(f.col('CODIGO_FILIAL'), f.col('TICKET'), f.col('DATA_VENDA')))    
    )
    
    if new_data_count > 0:
        logger.info('Writing hash table new data ')
        logger.info('Writing ' + str(new_data_count) + ' new data')
        writeNewData(spark, df_table_hash, silver_path, 'delta')
        
        logger.info('Writing restrited table new data')
        logger.info('Writing ' + str(new_data_count) + ' new data')
        writeNewData(spark, df_table_restricted, silver_path_restricted, 'delta')
        
    if updating_data_count > 0:
        logger.info('Updating hash table data')
        logger.info('Updating ' + str(updating_data_count) + ' data')
        updateNewData(spark, df_table_hash, silver_path, ['pk_single'])
            
        logger.info('Updanting restrited table data')
        logger.info('Updating ' + str(updating_data_count) + ' data')
        updateNewData(spark, df_table_restricted, silver_path_restricted, ['pk_single'])
        
    if deleting_data_count > 0:
        logger.info('Deleting hash table data')
        logger.info('Deleting ' + str(deleting_data_count) + ' data')
        deleteNewData(spark, silver_path)
        
        logger.info('Deleting restrited table data')
        logger.info('Deleting ' + str(deleting_data_count) + ' data')
        deleteNewData(spark, silver_path_restricted)

2022-03-22 12:35:39.265 | INFO     | __main__:<module>:2 - Defining restricted table
2022-03-22 12:35:39.332 | INFO     | __main__:<module>:11 - Defining hash table
2022-03-22 12:35:39.401 | INFO     | __main__:<module>:20 - Writing hash table new data 
2022-03-22 12:35:39.403 | INFO     | __main__:<module>:21 - Writing 21136 new data
2022-03-22 12:37:42.703 | INFO     | __main__:<module>:24 - Writing restrited table new data
2022-03-22 12:37:42.704 | INFO     | __main__:<module>:25 - Writing 21136 new data
2022-03-22 12:39:36.752 | INFO     | __main__:<module>:29 - Updating hash table data
2022-03-22 12:39:36.753 | INFO     | __main__:<module>:30 - Updating 321 data
2022-03-22 12:50:04.329 | INFO     | __main__:<module>:33 - Updanting restrited table data
2022-03-22 12:50:04.351 | INFO     | __main__:<module>:34 - Updating 321 data


In [ ]:
##################################
### Verificar está funcionando ###
##################################

In [ ]:
df_silver_hash_t = (
    spark
    .read
    .format("delta")
    .load(silver_path)
)

df_silver_restricted_t = (
    spark
    .read
    .format("delta")
    .load(silver_path_restricted)
)

In [ ]:
df_silver_hash_t.join(df_silver_restricted_t,df_silver_hash_t.PEDIDO == df_silver_restricted_t.PEDIDO,"inner").limit(100).toPandas()

In [26]:
df_pd = df_silver_teste.toPandas()

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:186: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series


In [27]:
df_pd

offset         timestamp_kafka  year  month  day  hour       PRODUTO  \
0      -24161 2022-03-09 13:45:47.668  2022      3    9    13  012N           
1       -7625 2022-03-09 13:45:04.422  2022      3    9    13  014V           
2       -7721 2022-03-09 13:45:04.425  2022      3    9    13  015W           
3      -36316 2022-03-09 13:46:14.155  2022      3    9    13  01YP           
4      -24283 2022-03-09 13:45:47.681  2022      3    9    13  0295           
...       ...                     ...   ...    ...  ...   ...           ...   
97900   -7864 2022-03-09 13:45:06.460  2022      3    9    13  ZMBD           
97901  -42646 2022-03-09 13:46:33.061  2022      3    9    13  ZU6N           
97902  -48612 2022-03-09 13:47:00.239  2022      3    9    13  ZU99           
97903   -9941 2022-03-09 13:45:13.131  2022      3    9    13  ZUJA           
97904  -39683 2022-03-09 13:46:26.686  2022      3    9    13  ZUL9           

      CODIGO_PRECO MATERIAL PERIODO_PCP  ... HR_B2C_CATEGORIA  ID_CEST_NCM  \
0                      None        None  ...             None          NaN   
1                      None        None  ...             None          NaN   
2                      None        None  ...             None          NaN   
3                      None        None  ...             None          NaN   
4                      None        None  ...             None          NaN   
...            ...      ...         ...  ...              ...          ...   
97900                  None        None  ...             None          NaN   
97901                  None        None  ...             None          NaN   
97902                  None        None  ...             None          NaN   
97903                  None        None  ...             None          NaN   
97904                  None        None  ...             None          NaN   

      HR_INDICA_BRINDE HR_PARCEIRO HR_ESTRATEGIA INFO_MODELO __op   __table  \
0                 None        None          None        None    r  PRODUTOS   
1                 None        None          None        None    r  PRODUTOS   
2                 None        None          None        None    r  PRODUTOS   
3                 None        None          None        None    r  PRODUTOS   
4                 None        None          None        None    r  PRODUTOS   
...                ...         ...           ...         ...  ...       ...   
97900             None        None          None        None    r  PRODUTOS   
97901             None        None          None        None    r  PRODUTOS   
97902             None        None          None        None    r  PRODUTOS   
97903             None        None          None        None    r  PRODUTOS   
97904             None        None          None        None    r  PRODUTOS   

      __source_ts_ms __deleted  
0      1646833546855     false  
1      1646833504018     false  
2      1646833504065     false  
3      1646833573323     false  
4      1646833547233     false  
...              ...       ...  
97900  1646833504834     false  
97901  1646833592148     false  
97902  1646833618534     false  
97903  1646833512635     false  
97904  1646833584853     false  

[97905 rows x 165 columns]

In [30]:
df_pd[df_pd["__op"] != "r"]

offset         timestamp_kafka  year  month  day  hour       PRODUTO  \
34      52663 2022-03-16 04:41:02.566  2022      3   16     4  06AK           
130     27191 2022-03-11 02:46:38.729  2022      3   11     2  4A8Y           
147     30130 2022-03-11 04:40:54.406  2022      3   11     4  4E25           
160     50758 2022-03-15 17:50:28.414  2022      3   15    17  4FPQ           
174     78439 2022-03-17 05:35:15.868  2022      3   17     5  54NA           
...       ...                     ...   ...    ...  ...   ...           ...   
97735   28635 2022-03-11 04:40:54.389  2022      3   11     4  H98D           
97736      59 2022-03-08 15:27:34.147  2022      3    8    15  H9CQ           
97757   77726 2022-03-17 05:44:28.417  2022      3   17     5  KFKN           
97794   64643 2022-03-16 17:50:57.216  2022      3   16    17  M06H           
97831   28760 2022-03-11 04:40:54.404  2022      3   11     4  UBL0           

      CODIGO_PRECO MATERIAL PERIODO_PCP  ... HR_B2C_CATEGORIA  ID_CEST_NCM  \
34                     None        None  ...             None          NaN   
130                    None        None  ...             None          NaN   
147                    None        None  ...             None          NaN   
160                    None        None  ...             None          NaN   
174                    None        None  ...         INFANTIL          NaN   
...            ...      ...         ...  ...              ...          ...   
97735                  None        None  ...             None          NaN   
97736                  None        None  ...             None          NaN   
97757                  None        None  ...             None          NaN   
97794                  None        None  ...             None          NaN   
97831                  None        None  ...             None          NaN   

      HR_INDICA_BRINDE HR_PARCEIRO HR_ESTRATEGIA INFO_MODELO __op   __table  \
34                None        None          None        None    u  PRODUTOS   
130               None        None          None        None    u  PRODUTOS   
147               None        None          None        None    u  PRODUTOS   
160               None        None          None        None    u  PRODUTOS   
174               None        None          None        None    u  PRODUTOS   
...                ...         ...           ...         ...  ...       ...   
97735             None        None          None        None    u  PRODUTOS   
97736             None        None          None        None    u  PRODUTOS   
97757             None        None          None        None    u  PRODUTOS   
97794             None        None          None        None    u  PRODUTOS   
97831             None        None          None        None    u  PRODUTOS   

      __source_ts_ms __deleted  
34     1647405657487     false  
130    1646966792393     false  
147    1646973646740     false  
160    1647366624380     false  
174    1647495310903     false  
...              ...       ...  
97735  1646973646740     false  
97736  1646753247243     false  
97757  1647495865523     false  
97794  1647453051560     false  
97831  1646973646740     false  

[1975 rows x 165 columns]